<a href="https://cognitiveclass.ai"><img src = "https://ibm.box.com/shared/static/9gegpsmnsoo25ikkbl4qzlvlyjbgxs5x.png" width = 400> </a>

<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>


## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1.  <a href="#item1">Download and Explore Dataset</a>

2.  <a href="#item2">Explore Neighborhoods in Toronto</a>

3.  <a href="#item3">Analyze Each Neighborhood</a>

4.  <a href="#item4">Cluster Neighborhoods</a>

5.  <a href="#item5">Examine Clusters</a>  
    </font>
    </div>


Download all the dependencies that we will need.


In [7]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

import collections

print('Libraries imported.')

Libraries imported.


<a id='item1'></a>


## 1. Download and Explore Dataset


We only manage to find some neighborhoods in Toronto for the project.


In [11]:
NeighbourhoodInDistrict = collections.namedtuple('NeighbourhoodInDistrict', ['district', 'neighbourhood'])
neighbourhood_tuples = []

filename = "toronto_neighbourhoods_list.txt"
with open(filename, 'r') as f:
    district = "INVALID"
    for line in f:
        if line[:2] == '--':
            district = line[2:-1] # exclude '\n'
            continue
        neighbourhood = line.split(',')[0]
        if neighbourhood[-1] == '\n':
            neighbourhood = neighbourhood[:-1]
        print('D: {}, N: {}'.format(district, neighbourhood))
        neighbourhood_tuples.append(NeighbourhoodInDistrict(district, neighbourhood))

D: Old Toronto, N: Alexandra Park
D: Old Toronto, N: The Annex
D: Old Toronto, N: Baldwin Village
D: Old Toronto, N: Cabbagetown
D: Old Toronto, N: CityPlace
D: Old Toronto, N: Chinatown
D: Old Toronto, N: Church and Wellesley
D: Old Toronto, N: Corktown
D: Old Toronto, N: Discovery District
D: Old Toronto, N: Distillery District
D: Old Toronto, N: Entertainment District
D: Old Toronto, N: East Bayfront
D: Old Toronto, N: Fashion District
D: Old Toronto, N: Financial District
D: Old Toronto, N: Garden District
D: Old Toronto, N: Grange Park
D: Old Toronto, N: Harbord Village
D: Old Toronto, N: Harbourfront
D: Old Toronto, N: Kensington Market
D: Old Toronto, N: First Chinatown
D: Old Toronto, N: Moss Park
D: Old Toronto, N: Old Town
D: Old Toronto, N: Quayside
D: Old Toronto, N: Queen Street West
D: Old Toronto, N: Regent Park
D: Old Toronto, N: South Core
D: Old Toronto, N: St. James Town
D: Old Toronto, N: St. Lawrence
D: Old Toronto, N: Toronto Islands
D: Old Toronto, N: Trefann Cou

#### Tranform the data into a _pandas_ dataframe


The next task is essentially transforming this data of nested Python dictionaries into a _pandas_ dataframe. So let's start by creating an empty dataframe.


In [ ]:
geolocator = Nominatim(user_agent="toronto_explorer")
# define the dataframe columns
column_names = ['District', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)
for nt in neighbourhood_tuples:
    address = '{}, {}, Toronto, Ontario'.format(nt.neighbourhood, nt.district)
    location = geolocator.geocode(address)
    if not location:
        print('cannot find latitude and longitude for "{}"'.format(address))
        continue
    latitude = location.latitude
    longitude = location.longitude
    neighborhoods = neighborhoods.append({'District': nt.district,
                                          'Neighborhood': nt.neighbourhood,
                                          'Latitude': latitude,
                                          'Longitude': longitude}, ignore_index=True)

cannot find latitude and longitude for "East Bayfront, Old Toronto, Toronto, Ontario"
cannot find latitude and longitude for "Ordinance Triangle, Old Toronto, Toronto, Ontario"


Quickly examine the resulting dataframe.


In [ ]:
neighborhoods.head()

And make sure that the dataset has all 5 boroughs and 306 neighborhoods.


In [115]:
print('The dataframe has {} neighborhoods.'.format(
        neighborhoods.shape[0]
    )
)

The dataframe has 89 neighborhoods.


#### Use geopy library to get the latitude and longitude values of New York City.


In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>ny_explorer</em>, as shown below.


In [116]:
location = geolocator.geocode('Toronto')
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### Create a map of Toronto with neighborhoods superimposed on top.


In [119]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Neighborhood']):
    label = neighborhood
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.


#### Define Foursquare Credentials and Version


In [118]:
CLIENT_ID = "5ZYZWNP1XIE3MUZKFCASPSJIOR13DKAS3KVMOGZXH4WQ15R3" # your Foursquare ID
CLIENT_SECRET = "D4K1ZKCUYZBKUEQOJWAHDGRHHMI3ZPZFFE0XHCSBFZKYOVMD" # your Foursquare Secret
ACCESS_TOKEN = "DABR2LI51C32L4KAC2BZYTG43U4TEYOYS2EBYQQAEOOB5TTD" # your FourSquare Access Token
VERSION = '20210223' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5ZYZWNP1XIE3MUZKFCASPSJIOR13DKAS3KVMOGZXH4WQ15R3
CLIENT_SECRET:D4K1ZKCUYZBKUEQOJWAHDGRHHMI3ZPZFFE0XHCSBFZKYOVMD


#### Let's explore the first neighborhood in our dataframe.


Get the neighborhood's name.


In [120]:
neighborhood_name = neighborhoods.loc[0, 'Neighborhood'] # neighborhood name
neighborhood_name

'Agincourt'

Get the neighborhood's latitude and longitude values.


In [122]:
neighborhood_latitude = neighborhoods.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighborhoods.loc[0, 'Longitude'] # neighborhood longitude value

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Agincourt are 43.7853531, -79.2785494.


#### Now, let's get the top 100 venues that are in Agincourt within a radius of 500 meters.


First, let's create the GET request URL. Name your URL **url**.


In [123]:
url = 'https://api.foursquare.com/v2/venues/search'
params = dict(
client_id=CLIENT_ID,
client_secret=CLIENT_SECRET,
v=VERSION,
ll='{},{}'.format(neighborhood_latitude, neighborhood_longitude),
oauth_token=ACCESS_TOKEN,
radius=500,
limit=LIMIT
)
params

{'client_id': '5ZYZWNP1XIE3MUZKFCASPSJIOR13DKAS3KVMOGZXH4WQ15R3',
 'client_secret': 'D4K1ZKCUYZBKUEQOJWAHDGRHHMI3ZPZFFE0XHCSBFZKYOVMD',
 'v': '20210223',
 'll': '43.7853531,-79.2785494',
 'oauth_token': 'DABR2LI51C32L4KAC2BZYTG43U4TEYOYS2EBYQQAEOOB5TTD',
 'radius': 500,
 'limit': 100}

Send the GET request and examine the resutls


In [125]:
results = requests.get(url=url, params=params).json()
results['response']['venues'][:5]

[{'id': '5058e50ce4b0a3bd556ccbdf',
  'name': 'Tim Hortons',
  'location': {'address': '4200 Sheppard Ave East',
   'crossStreet': 'Midland Ave',
   'lat': 43.78563681034057,
   'lng': -79.27921459078789,
   'labeledLatLngs': [{'label': 'display',
     'lat': 43.78563681034057,
     'lng': -79.27921459078789}],
   'distance': 62,
   'postalCode': 'M1S 1T3',
   'cc': 'CA',
   'city': 'Scarborough',
   'state': 'ON',
   'country': 'Canada',
   'formattedAddress': ['4200 Sheppard Ave East (Midland Ave)',
    'Scarborough ON M1S 1T3',
    'Canada']},
  'categories': [{'id': '4bf58dd8d48988d1e0931735',
    'name': 'Coffee Shop',
    'pluralName': 'Coffee Shops',
    'shortName': 'Coffee Shop',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
     'suffix': '.png'},
    'primary': True}],
  'referralId': 'v-1614140446',
  'hasPerk': False},
 {'id': '4f66441be4b0cded5e234b44',
  'name': 'Bus Stop 57, 85 & 190',
  'location': {'address': 'Midland & Shepard',
  

From the Foursquare lab in the previous module, we know that all the information is in the _response.venues_ key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.


In [126]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a _pandas_ dataframe.


In [127]:
# venues = results['response']['groups'][0]['items']
venues = results['response']['venues']  
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['name', 'categories', 'location.lat', 'location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng,categories
0,Tim Hortons,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",43.785637,-79.279215,Coffee Shop
1,"Bus Stop 57, 85 & 190","[{'id': '4bf58dd8d48988d12b951735', 'name': 'B...",43.785114,-79.278564,Bus Line
2,Lucky Fishing Tackle,"[{'id': '4bf58dd8d48988d1ff941735', 'name': 'M...",43.785550,-79.277799,Miscellaneous Shop
3,Midtown Plaza,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",43.784840,-79.277773,Shopping Mall
4,May Yan Seafood Restaurant 陸福海鮮酒家,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",43.784513,-79.277735,Chinese Restaurant


And how many venues were returned by Foursquare?


In [128]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


<a id='item2'></a>


## 2. Explore Neighborhoods in Manhattan


#### Let's create a function to repeat the same process to all the neighborhoods in Manhattan


In [130]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):  
    venue_lists = []
    for name, lat, lng in zip(names, latitudes, longitudes):
        print('Getting nearby venues for neighborhood {}...'.format(name))
            
        """ create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        """
        url = 'https://api.foursquare.com/v2/venues/explore'
        params = dict(
            client_id=CLIENT_ID,
            client_secret=CLIENT_SECRET,
            v=VERSION,
            ll='{},{}'.format(lat, lng),
            radius=radius,
            limit=LIMIT,
        )
            
        # make the GET request
        results = requests.get(url=url, params=params).json()
        venue_lists.append(get_venue_list(results, name, lat, lng))

    nearby_venues = pd.DataFrame([item for venue_list in venue_lists for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return nearby_venues

def get_venue_list(results, name, lat, lng):
    response = results["response"]
    if 'venues' in response:
        venues = response['venues']
        # return only relevant information for each nearby venue
        return [(
            name, 
            lat, 
            lng, 
            v['name'], 
            v['location']['lat'], 
            v['location']['lng'],  
            get_category_type(v)) for v in venues]
    else:
        venues = response['groups'][0]['items']
        # return only relevant information for each nearby venue
        return [(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in venues]

#### Now write the code to run the above function on each neighborhood and create a new dataframe called _manhattan_venues_.


In [132]:
# type your answer here
filtered_columns = ['Neighborhood', 'Latitude', 'Longitude']
names, latitudes, longitudes = neighborhoods.loc[:, filtered_columns].T.values
toronto_venues = getNearbyVenues(names, latitudes, longitudes)

Getting nearby venues for neighborhood Agincourt...
Getting nearby venues for neighborhood Alderwood...
Getting nearby venues for neighborhood Alexandra Park...
Getting nearby venues for neighborhood Allenby...
Getting nearby venues for neighborhood Amesbury...
Getting nearby venues for neighborhood Bedford Park...
Getting nearby venues for neighborhood Bermondsey...
Getting nearby venues for neighborhood Black Creek...
Getting nearby venues for neighborhood Bridle Path...
Getting nearby venues for neighborhood Bridlewood...
Getting nearby venues for neighborhood Brown's Corners...
Getting nearby venues for neighborhood Cabbagetown...
Getting nearby venues for neighborhood Cedarvale...
Getting nearby venues for neighborhood Chinatown...
Getting nearby venues for neighborhood CityPlace...
Getting nearby venues for neighborhood Clairville...
Getting nearby venues for neighborhood Cliffside...
Getting nearby venues for neighborhood Corktown...
Getting nearby venues for neighborhood Davenp

#### Let's check the size of the resulting dataframe


In [133]:
print(toronto_venues.shape)
toronto_venues.head()

(2508, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Agincourt,43.785353,-79.278549,One2 Snacks,43.787048,-79.276658,Asian Restaurant
1,Agincourt,43.785353,-79.278549,In Cheon House Korean & Japanese Restaurant 인천관,43.786468,-79.275693,Korean Restaurant
2,Agincourt,43.785353,-79.278549,Tim Hortons,43.785637,-79.279215,Coffee Shop
3,Agincourt,43.785353,-79.278549,Beef Noodle Restaurant 老李牛肉麵,43.785937,-79.276031,Chinese Restaurant
4,Agincourt,43.785353,-79.278549,大泉港式快餐 Great Fountain Fast Food,43.786835,-79.277400,Food Court


Let's check how many venues were returned for each neighborhood


In [138]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,12,12,12,12,12,12
Alderwood,7,7,7,7,7,7
Alexandra Park,100,100,100,100,100,100
Allenby,8,8,8,8,8,8
Amesbury,6,6,6,6,6,6
Bedford Park,2,2,2,2,2,2
Bermondsey,10,10,10,10,10,10
Black Creek,5,5,5,5,5,5
Bridle Path,2,2,2,2,2,2


#### Let's find out how many unique categories can be curated from all the returned venues


In [140]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 283 uniques categories.


<a id='item3'></a>


## 3. Analyze Each Neighborhood


In [137]:
# one hot encoding
onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
onehot = onehot[fixed_columns]

onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bike Trail,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hong Kong Restaurant,Hookah Bar,Hospital,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Outdoor Supply Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pide Place,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skating Rink,Ski Area,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Storage Facility,Street Art,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Syrian Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tech Startup,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Tibetan Restaurant,Tourist Information Center,Toy / Game Store,Trail,Train,Train Station,Tram Station,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game 

And let's examine the new dataframe size.


In [142]:
onehot.shape

(2508, 283)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category


In [143]:
grouped = onehot.groupby('Neighborhood').mean().reset_index()
grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bike Trail,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hong Kong Restaurant,Hookah Bar,Hospital,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Outdoor Supply Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pide Place,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skating Rink,Ski Area,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Storage Facility,Street Art,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Syrian Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tech Startup,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Tibetan Restaurant,Tourist Information Center,Toy / Game Store,Trail,Train,Train Station,Tram Station,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game 

#### Let's confirm the new size


In [94]:
grouped.shape

(40, 335)

#### Let's print each neighborhood along with the top 5 most common venues


In [144]:
num_top_venues = 5

for hood in grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = grouped[grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                   venue  freq
0     Chinese Restaurant  0.33
1             Food Court  0.08
2   Hong Kong Restaurant  0.08
3  Vietnamese Restaurant  0.08
4   Cantonese Restaurant  0.08


----Alderwood----
          venue  freq
0   Pizza Place  0.29
1           Pub  0.14
2           Gym  0.14
3  Skating Rink  0.14
4   Coffee Shop  0.14


----Alexandra Park----
                    venue  freq
0                     Bar  0.10
1                    Café  0.05
2  Furniture / Home Store  0.05
3    Caribbean Restaurant  0.05
4             Coffee Shop  0.03


----Allenby----
                venue  freq
0       Big Box Store  0.12
1        Intersection  0.12
2  African Restaurant  0.12
3          Restaurant  0.12
4   Fish & Chips Shop  0.12


----Amesbury----
                venue  freq
0        Intersection  0.17
1  Athletics & Sports  0.17
2         Gas Station  0.17
3         Coffee Shop  0.17
4                Park  0.17


----Bedford Park----
                           venu

#### Let's put that into a _pandas_ dataframe


First, let's write a function to sort the venues in descending order.


In [145]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.


In [147]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = grouped['Neighborhood']

for ind in np.arange(grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Chinese Restaurant,Food Court,Hong Kong Restaurant,Vietnamese Restaurant,Cantonese Restaurant,Korean Restaurant,Train Station,Coffee Shop,Asian Restaurant,Yoga Studio
1,Alderwood,Pizza Place,Pub,Gym,Skating Rink,Coffee Shop,Sandwich Place,Yoga Studio,Outdoor Supply Store,Organic Grocery,Optical Shop
2,Alexandra Park,Bar,Café,Furniture / Home Store,Caribbean Restaurant,Coffee Shop,Liquor Store,Gym / Fitness Center,Taco Place,Dessert Shop,Boutique
3,Allenby,Big Box Store,Intersection,African Restaurant,Restaurant,Fish & Chips Shop,Bookstore,Café,Fast Food Restaurant,Yoga Studio,Paper / Office Supplies Store
4,Amesbury,Intersection,Athletics & Sports,Gas Station,Coffee Shop,Park,Bank,Noodle House,North Indian Restaurant,Office,Opera House


<a id='item4'></a>


## 4. Cluster Neighborhoods


Run _k_-means to cluster the neighborhood into 5 clusters.


In [148]:
grouped.head()

NameError: name 'manhattan_grouped' is not defined

In [150]:
# set number of clusters
kclusters = 5

grouped_clustering = grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 3, 0, 0, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.


In [151]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

merged = neighborhoods

# merge grouped with neighborhoods to add latitude/longitude for each neighborhood
merged = merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

merged.head() # check the last columns!

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,43.785353,-79.278549,0,Chinese Restaurant,Food Court,Hong Kong Restaurant,Vietnamese Restaurant,Cantonese Restaurant,Korean Restaurant,Train Station,Coffee Shop,Asian Restaurant,Yoga Studio
1,Alderwood,43.601717,-79.545232,0,Pizza Place,Pub,Gym,Skating Rink,Coffee Shop,Sandwich Place,Yoga Studio,Outdoor Supply Store,Organic Grocery,Optical Shop
2,Alexandra Park,43.650787,-79.404318,0,Bar,Café,Furniture / Home Store,Caribbean Restaurant,Coffee Shop,Liquor Store,Gym / Fitness Center,Taco Place,Dessert Shop,Boutique
3,Allenby,43.711351,-79.553424,0,Big Box Store,Intersection,African Restaurant,Restaurant,Fish & Chips Shop,Bookstore,Café,Fast Food Restaurant,Yoga Studio,Paper / Office Supplies Store
4,Amesbury,43.706162,-79.483492,0,Intersection,Athletics & Sports,Gas Station,Coffee Shop,Park,Bank,Noodle House,North Indian Restaurant,Office,Opera House


Finally, let's visualize the resulting clusters


In [153]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged['Latitude'], merged['Longitude'], merged['Neighborhood'], merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>


## 5. Examine Clusters


Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.


#### Cluster 1


In [154]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

NameError: name 'manhattan_merged' is not defined

#### Cluster 2


In [107]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,Chinese Restaurant,Bakery,Dessert Shop,Hotpot Restaurant,American Restaurant,Cocktail Bar,Spa,Salon / Barbershop,Boutique,Ice Cream Shop
6,Central Harlem,African Restaurant,American Restaurant,Bar,Seafood Restaurant,French Restaurant,Cosmetics Shop,Chinese Restaurant,Cafeteria,Spa,Tapas Restaurant
8,Upper East Side,Italian Restaurant,Exhibit,Coffee Shop,Bakery,Gym / Fitness Center,Spa,Pizza Place,Juice Bar,Hotel,French Restaurant
10,Lenox Hill,Italian Restaurant,Cocktail Bar,Coffee Shop,Pizza Place,Sushi Restaurant,Burger Joint,Gym,Gym / Fitness Center,Café,Salon / Barbershop
14,Clinton,Italian Restaurant,Gym / Fitness Center,Theater,Sandwich Place,American Restaurant,Wine Shop,Coffee Shop,Spa,Cocktail Bar,Gym
15,Midtown,Hotel,Sporting Goods Shop,Theater,Clothing Store,Steakhouse,Bookstore,Bakery,Coffee Shop,American Restaurant,Spa
16,Murray Hill,Coffee Shop,Sandwich Place,Japanese Restaurant,Hotel,Bar,Burger Joint,American Restaurant,Gym / Fitness Center,Pizza Place,Jewish Restaurant
17,Chelsea,Coffee Shop,Bakery,Art Gallery,Italian Restaurant,Hotel,French Restaurant,Seafood Restaurant,Ice Cream Shop,Wine Shop,American Restaurant
19,East Village,Bar,Pizza Place,Korean Restaurant,Salon / Barbershop,Coffee Shop,Cocktail Bar,Ice Cream Shop,Italian Restaurant,Wine Bar,Speakeasy
20,Lower East Side,Chinese Restaurant,Latin American Restaurant,Bakery,Art Gallery,Café,Pizza Place,Yoga Studio,Pet Café,Mediterranean Restaurant,Thai Restaurant


#### Cluster 3


In [108]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Washington Heights,Café,Bakery,Mobile Phone Shop,Bank,Grocery Store,Park,Pizza Place,Latin American Restaurant,New American Restaurant,Gym
5,Manhattanville,Coffee Shop,Seafood Restaurant,Italian Restaurant,Deli / Bodega,Sushi Restaurant,Mexican Restaurant,Burger Joint,Bus Station,Supermarket,Café
9,Yorkville,Italian Restaurant,Gym,Coffee Shop,Deli / Bodega,Sushi Restaurant,Bar,Wine Shop,Mexican Restaurant,Japanese Restaurant,Bagel Shop
11,Roosevelt Island,Park,Baseball Field,Gym / Fitness Center,Gym,Supermarket,Greek Restaurant,Bubble Tea Shop,Soccer Field,Outdoors & Recreation,Food & Drink Shop
12,Upper West Side,Café,Italian Restaurant,Bar,Bakery,Wine Bar,Seafood Restaurant,Dessert Shop,French Restaurant,Breakfast Spot,Ice Cream Shop
13,Lincoln Square,Café,Plaza,Theater,Gym / Fitness Center,Concert Hall,Performing Arts Venue,Italian Restaurant,Wine Shop,Indie Movie Theater,Bakery
18,Greenwich Village,Italian Restaurant,Clothing Store,Sushi Restaurant,Café,American Restaurant,Coffee Shop,Boutique,Gym,Indian Restaurant,Bakery
21,Tribeca,Park,Italian Restaurant,American Restaurant,Wine Bar,Café,Spa,Coffee Shop,Scenic Lookout,Hotel,Boutique
23,Soho,Clothing Store,Italian Restaurant,Coffee Shop,Mediterranean Restaurant,Bakery,Boutique,Furniture / Home Store,Sporting Goods Shop,Café,Seafood Restaurant
24,West Village,Italian Restaurant,Cocktail Bar,New American Restaurant,American Restaurant,Park,Wine Bar,Theater,Coffee Shop,Cosmetics Shop,Gourmet Shop


#### Cluster 4


In [109]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,Stuyvesant Town,Park,Coffee Shop,Bar,Pet Service,German Restaurant,Fountain,Gym / Fitness Center,Skating Rink,Harbor / Marina,Heliport


#### Cluster 5


In [110]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Discount Store,Gym,Coffee Shop,Sandwich Place,Diner,Steakhouse,Shopping Mall,Seafood Restaurant,Deli / Bodega,Yoga Studio


### Thank you for completing this lab!

This notebook was created by [Alex Aklson](https://www.linkedin.com/in/aklson?cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork-21253531&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork-21253531&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ) and [Polong Lin](https://www.linkedin.com/in/polonglin?cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork-21253531&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork-21253531&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ). I hope you found this lab interesting and educational. Feel free to contact us if you have any questions!


This notebook is part of a course on **Coursera** called _Applied Data Science Capstone_. If you accessed this notebook outside the course, you can take this course online by clicking [here](http://cocl.us/DP0701EN_Coursera_Week3_LAB2).


## Change Log

| Date (YYYY-MM-DD) | Version | Changed By    | Change Description         |
| ----------------- | ------- | ------------- | -------------------------- |
| 2020-11-26        | 2.0     | Lakshmi Holla | Updated the markdown cells |
|                   |         |               |                            |
|                   |         |               |                            |

## <h3 align="center"> © IBM Corporation 2020. All rights reserved. <h3/>
